In [15]:
import numpy as np

In [1]:
import fasttext

In [12]:
def loadWordEmbedding(model):
    model = fasttext.load_model(model)
    vocab = ['unk']
    embd = [[0] * 100]
    for word in model.words:
        vocab.append(word)
        embd.append(model[word])
    print("loaded word2vec")
    return vocab, embd

In [ ]:
vocab, embd = loadWordEmbedding('model/fasttext.model.bin')
vocab_size = len(vocab)
embedding_dim = len(embd[0])
embedding = np.asarray(embd)

 **tensorflow load the trained word embedding (fasttext)**<br >
 return the vocab and embedding<br>
 the vocab:
 - 'unk',
 - '处会',
 - '倍觉',
 - '螳螂捕蝉',
 - '烟标',
 - '见证人',
 - '玩到',
 - '刑满释放',
 
the embedding:
- [[ 0.        ,  0.        ,  0.        , ...,  0.        ,
     0.        ,  0.        ],
- [ 0.20675033,  0.13941619, -0.02178967, ...,  0.08394553,
    -0.57525939, -0.17493315],
- [ 0.02645796, -0.16807704, -0.33025488, ...,  0.18632847,
    0.20377228,  0.1036511 ],

note: add the unk is that the word not in trained word embedding. and the vector is 0